In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
#Import basic libraries
%matplotlib inline

import os
import sys
import settings
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from django_pandas.io import read_frame
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates

from matplotlib.dates import date2num

# imports
import pandas_datareader.data as pdr
import datetime
import talib
from talib.abstract import *
from talib import MA_Type

# format price data
pd.options.display.float_format = '{:0.2f}'.format

## Pattern recognition on the OHLC data

In [ ]:
#Prepare to load stock data as pandas dataframe from source. In this case, prepare django
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from stocks.models import Listing, Stock

In [ ]:
import datetime
from pandas.tseries.frequencies import to_offset
def get_stock_listing(stock, duration=None, last_date = datetime.date.today(), studies=None, resample=False, monthly=False):
    requested_duration = duration
    if duration == None and studies==None:
        duration = 365
        requested_duration = duration
    elif duration == -1:
        duration = 3650
    elif studies is not None:
        #studies={'daily': ['rsi', 'ema20', 'ema10', 'sma200'],
        #                   'weekly':['rsi', 'ema20', 'ema10'],
        #                   'monthly': ['rsi']}
        if studies.get('monthly', None) is not None:
            if 'rsi' in studies.get('monthly'):
                duration = 500 #Need at least 14 months data for RSI
        elif studies.get('weekly', None) is not None:
            if 'rsi' in studies.get('weekly'):
                duration = 14*7 if duration is None else max(duration, 14*7)#Need at least 14 weeks data for RSI
            if 'ema20' in studies.get('weekly'):
                duration = 21*7 if duration is None else max(duration, 14*7)#Need at least 20 weeks data for EMA
            if 'ema10' in studies.get('weekly'):
                duration = 11*7 if duration is None else max(duration, 11*7)#Need at least 20 weeks data for EMA
        elif studies.get('daily', None) is not None:
            if 'rsi' in studies.get('daily'):
                duration = 14 if duration is None else max(duration, 14)#Need at least 14 days data for RSI
            if 'ema20' in studies.get('daily'):
                duration = 21 if duration is None else max(duration, 21)#Need at least 20 days data for EMA
            if 'ema10' in studies.get('daily'):
                duration = 11 if duration is None else max(duration, 11)#Need at least 10 days data for EMA
            if 'sma200' in studies.get('daily'):
                duration = 291 if duration is None else max(duration, 291)#Need at least 291 days data for SMA
                
    #print(duration)
    first_date = last_date - datetime.timedelta(days=duration)
    listing = Listing.objects.filter(stock=stock, date__range=(first_date, last_date))
    df = read_frame(listing, index_col='date')
    for column in df.columns:
        if column != 'stock':
           df[column] = pd.to_numeric(df[column])
    df = df.sort_index()
    df = df.reindex(columns = ['opening', 'high', 'low', 'closing', 'traded', 'deliverable', 'trades'])
    df.rename(columns={"opening": "open", 
                       "high": "high", 
                       "low": "low", 
                       "closing":"close", 
                       "traded":"volume", 
                       'deliverable':'delivery',
                       'trades':'trades'}, inplace=True)
    df.index = pd.to_datetime(df.index)

    #Delete duplicate columns
    #df = df.loc[:,~df.columns.duplicated()]
    df.drop_duplicates(inplace = True)
    df.dropna(inplace=True)
    if resample:
        #Resample weekly
        logic = {'open'  : 'first',
                 'high'  : 'max',
                 'low'   : 'min',
                 'close' : 'last',
                 'volume': 'sum',
                 'delivery': 'sum',
                 'trades': 'sum'}
        #Resample on weekly levels
        if monthly:
            df = df.resample('M').apply(logic)
        else:
            df = df.resample('W').apply(logic)
            df.index -= to_offset("6D")
        
    #Add the studies inplace 
    if studies is not None and studies.get('daily', None) is not None and len(df)>0:
        if 'rsi' in studies.get('daily'):
            df['rsi'] = talib.RSI(df['close'], 14)
        if 'ema20' in studies.get('daily'):
            df['ema20'] = talib.EMA(df['close'], 20)
        if 'ema10' in studies.get('daily'):
            df['ema10'] = talib.EMA(df['close'], 10)
        if 'sma200' in studies.get('daily'):
            df['sma200'] = talib.SMA(df['close'], 200)
    if studies is not None and studies.get('weekly', None) is not None and len(df)>0:
        if 'rsi' in studies.get('weekly'):
            df['rsi'] = talib.RSI(df['close'], 14)
        if 'ema20' in studies.get('daily'):
            df['ema20'] = talib.EMA(df['close'], 20)
        if 'ema10' in studies.get('daily'):
            df['ema10'] = talib.EMA(df['close'], 10)
    
            

    #print(df.tail())
    #Optionally, filter out by date range
    #start_date = '2020-01-01'
    #end_date = '2021-12-31'
    #df = df.loc[start_date:end_date]
    return df

In [ ]:
#https://github.com/CanerIrfanoglu/medium/blob/master/candle_stick_recognition/candle_rankings.py
#https://medium.com/analytics-vidhya/recognizing-over-50-candlestick-patterns-with-python-4f02a1822cb5
candle_rankings = {
        "CDL3LINESTRIKE_Bull": 1,
        "CDL3LINESTRIKE_Bear": 2,
        "CDL3BLACKCROWS_Bull": 3,
        "CDL3BLACKCROWS_Bear": 3,
        "CDLEVENINGSTAR_Bull": 4,
        "CDLEVENINGSTAR_Bear": 4,
        "CDLTASUKIGAP_Bull": 5,
        "CDLTASUKIGAP_Bear": 5,
        "CDLINVERTEDHAMMER_Bull": 6,
        "CDLINVERTEDHAMMER_Bear": 6,
        "CDLMATCHINGLOW_Bull": 7,
        "CDLMATCHINGLOW_Bear": 7,
        "CDLABANDONEDBABY_Bull": 8,
        "CDLABANDONEDBABY_Bear": 8,
        "CDLBREAKAWAY_Bull": 10,
        "CDLBREAKAWAY_Bear": 10,
        "CDLMORNINGSTAR_Bull": 12,
        "CDLMORNINGSTAR_Bear": 12,
        "CDLPIERCING_Bull": 13,
        "CDLPIERCING_Bear": 13,
        "CDLSTICKSANDWICH_Bull": 14,
        "CDLSTICKSANDWICH_Bear": 14,
        "CDLTHRUSTING_Bull": 15,
        "CDLTHRUSTING_Bear": 15,
        "CDLINNECK_Bull": 17,
        "CDLINNECK_Bear": 17,
        "CDL3INSIDE_Bull": 20,
        "CDL3INSIDE_Bear": 56,
        "CDLHOMINGPIGEON_Bull": 21,
        "CDLHOMINGPIGEON_Bear": 21,
        "CDLDARKCLOUDCOVER_Bull": 22,
        "CDLDARKCLOUDCOVER_Bear": 22,
        "CDLIDENTICAL3CROWS_Bull": 24,
        "CDLIDENTICAL3CROWS_Bear": 24,
        "CDLMORNINGDOJISTAR_Bull": 25,
        "CDLMORNINGDOJISTAR_Bear": 25,
        "CDLXSIDEGAP3METHODS_Bull": 27,
        "CDLXSIDEGAP3METHODS_Bear": 26,
        "CDLTRISTAR_Bull": 28,
        "CDLTRISTAR_Bear": 76,
        "CDLGAPSIDESIDEWHITE_Bull": 46,
        "CDLGAPSIDESIDEWHITE_Bear": 29,
        "CDLEVENINGDOJISTAR_Bull": 30,
        "CDLEVENINGDOJISTAR_Bear": 30,
        "CDL3WHITESOLDIERS_Bull": 32,
        "CDL3WHITESOLDIERS_Bear": 32,
        "CDLONNECK_Bull": 33,
        "CDLONNECK_Bear": 33,
        "CDL3OUTSIDE_Bull": 34,
        "CDL3OUTSIDE_Bear": 39,
        "CDLRICKSHAWMAN_Bull": 35,
        "CDLRICKSHAWMAN_Bear": 35,
        "CDLSEPARATINGLINES_Bull": 36,
        "CDLSEPARATINGLINES_Bear": 40,
        "CDLLONGLEGGEDDOJI_Bull": 37,
        "CDLLONGLEGGEDDOJI_Bear": 37,
        "CDLHARAMI_Bull": 38,
        "CDLHARAMI_Bear": 72,
        "CDLLADDERBOTTOM_Bull": 41,
        "CDLLADDERBOTTOM_Bear": 41,
        "CDLCLOSINGMARUBOZU_Bull": 70,
        "CDLCLOSINGMARUBOZU_Bear": 43,
        "CDLTAKURI_Bull": 47,
        "CDLTAKURI_Bear": 47,
        "CDLDOJISTAR_Bull": 49,
        "CDLDOJISTAR_Bear": 51,
        "CDLHARAMICROSS_Bull": 50,
        "CDLHARAMICROSS_Bear": 80,
        "CDLADVANCEBLOCK_Bull": 54,
        "CDLADVANCEBLOCK_Bear": 54,
        "CDLSHOOTINGSTAR_Bull": 55,
        "CDLSHOOTINGSTAR_Bear": 55,
        "CDLMARUBOZU_Bull": 71,
        "CDLMARUBOZU_Bear": 57,
        "CDLUNIQUE3RIVER_Bull": 60,
        "CDLUNIQUE3RIVER_Bear": 60,
        "CDL2CROWS_Bull": 61,
        "CDL2CROWS_Bear": 61,
        "CDLBELTHOLD_Bull": 62,
        "CDLBELTHOLD_Bear": 63,
        "CDLHAMMER_Bull": 65,
        "CDLHAMMER_Bear": 65,
        "CDLHIGHWAVE_Bull": 67,
        "CDLHIGHWAVE_Bear": 67,
        "CDLSPINNINGTOP_Bull": 69,
        "CDLSPINNINGTOP_Bear": 73,
        "CDLUPSIDEGAP2CROWS_Bull": 74,
        "CDLUPSIDEGAP2CROWS_Bear": 74,
        "CDLGRAVESTONEDOJI_Bull": 77,
        "CDLGRAVESTONEDOJI_Bear": 77,
        "CDLHIKKAKEMOD_Bull": 82,
        "CDLHIKKAKEMOD_Bear": 81,
        "CDLHIKKAKE_Bull": 85,
        "CDLHIKKAKE_Bear": 83,
        "CDLENGULFING_Bull": 84,
        "CDLENGULFING_Bear": 91,
        "CDLMATHOLD_Bull": 86,
        "CDLMATHOLD_Bear": 86,
        "CDLHANGINGMAN_Bull": 87,
        "CDLHANGINGMAN_Bear": 87,
        "CDLRISEFALL3METHODS_Bull": 94,
        "CDLRISEFALL3METHODS_Bear": 89,
        "CDLKICKING_Bull": 96,
        "CDLKICKING_Bear": 102,
        "CDLDRAGONFLYDOJI_Bull": 98,
        "CDLDRAGONFLYDOJI_Bear": 98,
        "CDLCONCEALBABYSWALL_Bull": 101,
        "CDLCONCEALBABYSWALL_Bear": 101,
        "CDL3STARSINSOUTH_Bull": 103,
        "CDL3STARSINSOUTH_Bear": 103,
        "CDLDOJI_Bull": 104,
        "CDLDOJI_Bear": 104,
        "CDLLONGLINE_Bull": 53,
        "CDLLONGLINE_Bear": 53,
        "CDLSHORTLINE_Bull": 85,
        "CDLSHORTLINE_Bear": 66,
        "CDLSTALLEDPATTERN_Bull": 93,
        "CDLSTALLEDPATTERN_Bear": 93,
        "CDLKICKINGBYLENGTH": 96,
        "CDLKICKINGBYLENGTH_Bear": 102
    }

## How does one determine the efficiency of an indicator/signal?

Currently, it seems hard that I could use the patterns alone in determining the trades (without understanding the patterns).
So, what I can do is:

1. Let the code run on daily basis and generate signals in the time frame (if daily, then for a month's data etc). 
   Print a summary of signals generated on the day, and manually evaluate them. (Lesser time to code, so we'll do that.)
2. Try to also estimate the accuracy/reliability of the candlestick myself rather than relying on someone else's 
   blackbox evaluation. The question is, how to deduce the efficacy (what time frame)? One stab at the answer would be, 
   understand each indicator and have its custom window determined empirically or via text.

In [ ]:
def evaluate_signal(df, signal_val, signal_frequency, success_frequency, tolerance=2):
    '''
    For the signal value passed, determine if price moves in the direction of signal 
    or not within the tolerance period and update the probability of success metric (frequentist)
    '''
    occurances = df[df[signal_val]!=0]
    #print(occurances.tail())
    idxs = []
    for idx, occurance in occurances.iterrows():
        #print(df.index.get_loc(idx))
        idxs.append(df.index.get_loc(idx))

    for idx in idxs:
        #print(idx)
        if df.iloc[idx][signal_val] > 0:
            #Bullish
            signal_frequency['bullish'][signal_val] += 1
            if idx+tolerance <= len(df)-1 and df.iloc[idx+tolerance]['close']> df.iloc[idx]['close']:
                #Pattern is success
                success_frequency['bullish'][signal_val] +=1
        elif df.iloc[idx][signal_val] < 0:
            #Bearish
            signal_frequency['bearish'][signal_val] += 1
            if idx+tolerance <= len(df)-1 and df.iloc[idx+tolerance]['close']< df.iloc[idx]['close']:
                #Pattern is success
                success_frequency['bearish'][signal_val] +=1
    

In [ ]:
#For all stocks, for all days, run the above function once to determine the score of each indicator
# create columns for each pattern
candle_names = talib.get_function_groups()['Pattern Recognition']

signal_frequency = {'bullish': {}, 'bearish':{}}
success_frequency = {'bullish': {}, 'bearish':{}}

for candle in candle_names:
    signal_frequency['bullish'][candle] = 0
    success_frequency['bullish'][candle] = 0
    signal_frequency['bearish'][candle] = 0
    success_frequency['bearish'][candle] = 0
    
#Asian paints
#stock = Stock.objects.get(sid='ASIANPAINT')
#listing = get_stock_listing(stock, duration=-1, last_date = datetime.date.today())
#for candle in candle_names:
#    listing[candle] = getattr(talib, candle)(listing['open'], listing['high'], listing['low'], listing['close'])
#    evaluate_signal(listing, candle, signal_frequency, success_frequency)
    
for stock in Stock.objects.all():
    #listing = get_stock_listing(stock, duration=-1, last_date = datetime.date.today(), resample=False)
    listing = get_stock_listing(stock, duration=-1, last_date = datetime.date.today(), resample=False, monthly=False) #For weekly/monthly charts
    if len(listing)==0:
        continue
    for candle in candle_names:
        listing[candle] = getattr(talib, candle)(listing['open'], listing['high'], listing['low'], listing['close'])
        evaluate_signal(listing, candle, signal_frequency, success_frequency, tolerance=2)

In [ ]:
print('Bullish')
for key in success_frequency['bullish']:
    if signal_frequency['bullish'][key] !=0:
        print(f"{key}: {success_frequency['bullish'][key]/signal_frequency['bullish'][key]}")
    else:
        print(f'{key}: NA')
print('Bearish')
for key in success_frequency['bearish']:
    if signal_frequency['bearish'][key] !=0:
        print(f"{key}: {success_frequency['bearish'][key]/signal_frequency['bearish'][key]}")
    else:
        print(f'{key}: NA')

In [ ]:
def detect_fractals(df):
    df['fractal'] = 0
    
    for i in range(2,df.shape[0]-2):
        if df['low'][i] < df['low'][i-1]  and df['low'][i] < df['low'][i+1] and df['low'][i+1] < df['low'][i+2] and df['low'][i-1] < df['low'][i-2]:
            #df['fractal'][i] = 1.0
            df.iloc[i, df.columns.get_loc('fractal')] = 1.0
        elif df['high'][i] > df['high'][i-1]  and df['high'][i] > df['high'][i+1] and df['high'][i+1] > df['high'][i+2] and df['high'][i-1] > df['high'][i-2]:
            #df['fractal'][i] = -1.0
            df.iat[i, df.columns.get_loc('fractal')] = -1.0
    return df

In [ ]:
#Test if it detects fractals properly
data = {'high': [1, 2, 3, 2, 1, 2, 3], 'low': [1, 2, 3, 2, 1, 2, 3]}  

testdf = pd.DataFrame(data)

testdf = detect_fractals(testdf)
testdf.head()

In [ ]:
#Compute efficacy of fractals patterns as well

signal_frequency = {'bullish': {}, 'bearish':{}}
success_frequency = {'bullish': {}, 'bearish':{}}

candle_names = ['fractal']
for candle in candle_names:
    signal_frequency['bullish'][candle] = 0
    success_frequency['bullish'][candle] = 0
    signal_frequency['bearish'][candle] = 0
    success_frequency['bearish'][candle] = 0
    
    
for stock in Stock.objects.all():
    #listing = get_stock_listing(stock, duration=-1, last_date = datetime.date.today(), resample=False)
    listing = get_stock_listing(stock, duration=-1, last_date = datetime.date.today(), resample=False, monthly=False) #For weekly/monthly charts
    if len(listing)==0:
        continue
    for candle in candle_names:
        df = detect_fractals(listing)
        evaluate_signal(listing, candle, signal_frequency, success_frequency, tolerance=3)
        
print('Bullish')
for key in success_frequency['bullish']:
    if signal_frequency['bullish'][key] !=0:
        print(f"{key}: {success_frequency['bullish'][key]/signal_frequency['bullish'][key]}")
    else:
        print(f'{key}: NA')
print('Bearish')
for key in success_frequency['bearish']:
    if signal_frequency['bearish'][key] !=0:
        print(f"{key}: {success_frequency['bearish'][key]/signal_frequency['bearish'][key]}")
    else:
        print(f'{key}: NA')

In [ ]:
print(signal_frequency['bullish']['fractal'])
print(success_frequency['bullish']['fractal'])

print(signal_frequency['bearish']['fractal'])
print(success_frequency['bearish']['fractal'])

In [ ]:
def get_trend(df):
    '''
    A stock is in some sort of trend on various time scales. 
    The hypothesis is that some chart patterns have a greater reliability in a particular trend.
    So, we want to combine trend and other signals to create higher probability of profit.
    
    We want to test out this hypothesis.
    '''
    pass
    
def get_volume_signals(df):
    '''
    Look at the trading volume and delivery volumes
    1. Check if unusually large amount of delivery volume is there (not percentages)
    2. Check if unusually large trading volume is there
    '''
    signals = {'volume_shoot_up': None,
               'volume_contract': None,
               'delivery_shoot_up': None,
               'delivery_contract': None,
               'delivery_ptage_shoot_up': None,
               'delivery_ptage_contract': None,
               'volume_per_trade_shoot_up': None,
               'volume_per_trade_contract': None,}
    period = 20 #20 day mean
    std = df['volume'].ewm(span=period).std()
    
    #std = df.iloc[-21:-1]['volume'].std()
    vol_ema = talib.EMA(df['volume'], period)

    if np.isnan(vol_ema.iloc[-2])==False and  std.iloc[-2] != 0 and df.iloc[-1]['volume'] >= (vol_ema.iloc[-2]+ std.iloc[-2]):
        signals['volume_shoot_up'] = (df.iloc[-1]['volume'] - vol_ema.iloc[-2])/std.iloc[-2]
    elif np.isnan(vol_ema.iloc[-2])==False and std.iloc[-2] != 0 and df.iloc[-1]['volume'] <= (vol_ema.iloc[-2]- std.iloc[-2]):
        signals['volume_contract'] = (vol_ema.iloc[-2] - df.iloc[-1]['volume'])/std.iloc[-2]
    #else:
    #    print(f"Volume: {df.iloc[-1]['volume']}  EMA: {vol_ema.iloc[-2]} STD: {std}")
        
    #std = df[-21:-1]['delivery'].std()
    std = df['delivery'].ewm(span=period).std()
    del_ema = talib.EMA(df['delivery'], period)
    if np.isnan(del_ema.iloc[-2])==False and std.iloc[-2] != 0 and df.iloc[-1]['delivery'] >= (del_ema.iloc[-2]+ std.iloc[-2]):
        signals['delivery_shoot_up'] = (df.iloc[-1]['delivery'] - del_ema.iloc[-2])/std.iloc[-2]
    elif np.isnan(del_ema.iloc[-2])==False and std.iloc[-2] != 0 and df.iloc[-1]['delivery'] <= (del_ema.iloc[-2]- std.iloc[-2]):
        signals['delivery_contract'] = (del_ema.iloc[-2] - df.iloc[-1]['delivery'])/std.iloc[-2]
    #else:
    #    print(f"Delivery: {df.iloc[-1]['delivery']}  EMA: {del_ema.iloc[-2]} STD: {std}")
    
    dl_ptage = df['delivery']/df['volume']
    dl_ema = talib.SMA(dl_ptage, period)
    #std = dl_ptage[-21:-1].std()
    std = dl_ptage.ewm(span=period).std()
    if np.isnan(dl_ema.iloc[-2])==False and std.iloc[-2] != 0 and dl_ptage.iloc[-1] >= (dl_ema.iloc[-2]+ std.iloc[-2]):
        signals['delivery_ptage_shoot_up'] = (dl_ptage.iloc[-1] - dl_ema.iloc[-2])/std.iloc[-2]
    elif np.isnan(dl_ema.iloc[-2])==False and std.iloc[-2] != 0 and dl_ptage.iloc[-1] <= (dl_ema.iloc[-2]- std.iloc[-2]):
        signals['delivery_ptage_contract'] = (dl_ema.iloc[-2] - dl_ptage.iloc[-1])/std.iloc[-2]
    #else:
    #    print(f"Delivery %: {dl_ptage.iloc[-1]}  EMA: {dl_ema.iloc[-2]} STD: {std}")
    
    vpt_ptage = df['volume']/df['trades'] #Volume per trade
    dl_ema = talib.SMA(vpt_ptage, period)
    #std = vpt_ptage[-21:-1].std()
    std = vpt_ptage.ewm(span=period).std()
    if np.isnan(dl_ema.iloc[-2])==False and std.iloc[-2] != 0 and vpt_ptage.iloc[-1] >= (dl_ema.iloc[-2]+ std.iloc[-2]):
        signals['volume_per_trade_shoot_up'] = (vpt_ptage.iloc[-1] - dl_ema.iloc[-2])/std.iloc[-2]
    elif np.isnan(dl_ema.iloc[-2])==False and std.iloc[-2] != 0 and vpt_ptage.iloc[-1] <= (dl_ema.iloc[-2]- std.iloc[-2]):
        signals['volume_per_trade_contract'] = (dl_ema.iloc[-2] - vpt_ptage.iloc[-1])/std.iloc[-2]
    #else:
    #    print(f"Volume Per trade %: {vpt_ptage.iloc[-1]}  EMA: {dl_ema.iloc[-2]} STD: {std}")
     
    
    return signals

In [ ]:
def compute_rrg(df_reference, df_target):
    '''
    Compute the relative rotation graph (RRG) values of df_target w.r.t. df_reference
    
    The idea is to find the sectors which are likely to go up, and then find the leaders in 
    that sector to improve the probability of profit.
    '''
    pass

In [ ]:
def get_last_day_patterns(df):
    candle_names = talib.get_function_groups()['Pattern Recognition']
    patterns = {'bullish': [], 'bearish': []}
    for candle in candle_names:
        df[candle] = getattr(talib, candle)(df['open'], df['high'], df['low'], df['close'])
        if df.iloc[-1][candle]>0:
            patterns['bullish'].append(candle)
        elif df.iloc[-1][candle]<0:
            patterns['bearish'].append(candle)
    df = detect_fractals(df)
    if df.iloc[-3]['fractal']>0:
        patterns['bullish'].append('fractal')
    elif df.iloc[-3]['fractal']<0:
        patterns['bearish'].append('fractal')
    #print(df.iloc[-1])
    return patterns

def get_signals(df, threshold = 0.05):
    overlap_indicators = ['ema20', 'ema10', 'sma200',
                  'w_ema20', 'w_ema10', 'w_sma200',]
    signals = {'proximity_short': [],
               'proximity_long': [],
               'price_crossover_short': [],
               'price_crossover_long': [],}
    for indicator in overlap_indicators:
        if indicator in df.columns:
            #If price is near the indicator (5%), flag the indicator
            if (abs(df.iloc[-1]['close'] - df.iloc[-1][indicator])/df.iloc[-1]['close'])<= threshold:
                if df.iloc[-1]['close'] < df.iloc[-1][indicator]:
                    signals['proximity_short'].append(indicator)
                if df.iloc[-1]['close'] > df.iloc[-1][indicator]:
                    signals['proximity_long'].append(indicator)
            #If price is crossing over the indicator, then flag too
            if df.iloc[-1]['low'] < df.iloc[-1][indicator] and df.iloc[-1]['high'] >= df.iloc[-1][indicator]:
                if df.iloc[-1]['close'] > df.iloc[-1]['open']:
                    signals['price_crossover_short'].append(indicator)
                else:
                    signals['price_crossover_long'].append(indicator)
                
    return signals

In [ ]:
def do_stuff(stock, prefix=''):
    listing = get_stock_listing(stock, duration=30, last_date = datetime.date.today(), 
                                studies={'daily': ['rsi', 'ema20', 'sma200'],})
                                         #'weekly':['rsi', 'ema20', 'ema10'],
                                         #'monthly': ['rsi']
                                        #})
    #print(listing.tail())
    if len(listing)==0:
        return
    #if listing.index[-1].date() != datetime.date.today() - datetime.timedelta(days=1):
    dateval = datetime.date.today()
    if datetime.date.today().weekday() in [5,6]: #If its saturday (5) or sunday (6)
        dateval = dateval - datetime.timedelta(days=abs(4-datetime.date.today()))
    if listing.index[-1].date() != dateval:
        print('No data for today on stock {}'.format(stock))
        return
    patterns = get_last_day_patterns(listing)
    signals = get_signals(listing)
    vol_sigs = get_volume_signals(listing)
    if len(patterns['bullish'])>0 or len(patterns['bearish'])>0:
        print(f"{prefix}{stock.sid} \nBullish: {patterns['bullish']}\tBearish: {patterns['bearish']}")
    else:
        print(f"{prefix}{stock.sid}: No patterns")
    if len(signals['proximity_short'])>0:
        print(f"Bearish Proximity signals: {signals['proximity_short']}")
    if len(signals['proximity_long'])>0:
        print(f"Bullish Proximity signals: {signals['proximity_long']}")
    if len(signals['price_crossover_short'])>0:
        print(f"Bearish Crossover signals: {signals['price_crossover_short']}")
    if len(signals['price_crossover_long'])>0:
        print(f"Bullish Crossover signals: {signals['price_crossover_long']}")
    for key in vol_sigs:
        if vol_sigs[key] is not None:
            print(f"{key}: {vol_sigs[key]}")

In [ ]:
#List of FnO stocks (where bullish and bearish signals are required)
fno = []
with open('fno.txt', 'r') as fd:
    for line in fd:
        fno.append(line.strip())
fno = sorted(fno)

for sname in fno:
    try:
        stock = Stock.objects.get(sid=sname)
        do_stuff(stock, prefix='[FnO]')
    except Stock.DoesNotExist:
        print(f'{sname} name not present')

In [ ]:
#Bearish scans for these too
portfolio = []
with open('portfolio.txt', 'r') as fd:
    for line in fd:
        portfolio.append(line.strip())
portfolio = sorted(portfolio)
for sname in portfolio:
    try:
        stock = Stock.objects.get(sid=sname)
        do_stuff(stock, prefix='[PF]')
    except Stock.DoesNotExist:
        print(f'{sname} name not present')

In [ ]:
margin_stocks = []
with open('margin.txt', 'r') as fd:
    for line in fd:
        margin_stocks.append(line.strip())
        
for stock in Stock.objects.all():
    #listing = get_stock_listing(stock, duration=30, last_date = datetime.date(2020, 12, 31))
    #print(stock)
    if (stock.sid in fno) or (stock.sid in portfolio):
        continue
    if stock.sid in margin_stocks:
        do_stuff(stock, prefix='[MG]')
    else:
        do_stuff(stock)

In [ ]:
stock = Stock.objects.get(sid='RAIN')

print (stock.sid)
print(stock.security)
print (stock)
listing = get_stock_listing(stock, duration=300, last_date = datetime.date.today())
listing.tail()

In [ ]:
from indicators import *
tolerance = 1 #days
df  = listing.rename(columns={"open": "Open", "high": "High", "low": "Low", "close":"Close", "volume":"Volume"})
divergences = autodetect_divergence(ohlc = df, indicator_data=talib.RSI(df['Close'], 14), tolerance = tolerance)

In [ ]:
#df.iloc[divergences.index[divergences['regularBull']>0] | divergences.index[divergences['hiddenBull']>0]]
df.iloc[divergences.index[divergences['regularBull']>0]]

In [ ]:
signals = get_volume_signals(listing)
print(signals)